In [35]:
import pickle
import os
import random
import argparse
import numpy as np
import pandas as pd
import torch
from glob import glob
from sklearn.metrics import accuracy_score

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, BertForSequenceClassification, Trainer, TrainingArguments, BertConfig
from load_data import *

In [2]:
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from torch.utils.data import DataLoader
from torch.optim import *
from tqdm import tqdm

In [3]:

def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
    }


### Train, Validation 나누기

In [4]:
seed_everything(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')

# load dataset
dataset = load_data("/opt/ml/input/data/train/train.tsv")
label = dataset['label'].values

dataset

,sentence,entity_01,entity_02,label
0,영국에서 사용되는 스포츠 유틸리티 자동차의 브랜드로는 랜드로버(Land Rover)...,랜드로버,자동차,17
1,"선거에서 민주당은 해산 전 의석인 230석에 한참 못 미치는 57석(지역구 27석,...",민주당,27석,0
2,유럽 축구 연맹(UEFA) 집행위원회는 2014년 1월 24일에 열린 회의를 통해 ...,유럽 축구 연맹,UEFA,6
3,"용병 공격수 챠디의 부진과 시즌 초 활약한 강수일의 침체, 시즌 중반에 영입한 세르...",강수일,공격수,2
4,람캄행 왕은 1237년에서 1247년 사이 수코타이의 왕 퍼쿤 씨 인트라팃과 쓰엉 ...,람캄행,퍼쿤 씨 인트라팃,8
...,...,...,...,...
8995,2002년 FIFA 월드컵 사우디아라비아와의 1차전에서 독일은 8-0으로 승리하였는...,사우디아라비아,2002년,0
8996,일본의 2대 메이커인 토요타와 닛산은 시장 점유율을 높이기 위한 신차 개발을 계속하...,토요타,일본,9
8997,방호의의 손자 방덕룡(方德龍)은 1588년(선조 21년) 무과에 급제하고 낙안군수로...,방덕룡,선무원종공신(宣武原從功臣),2
8998,LG전자는 올해 초 국내시장에 출시한 2020년형 ‘LG 그램’ 시리즈를 이달부터 ...,LG전자,북미,0


In [6]:
def tokenized_dataset(dataset, tokenizer):
    concat_entity = []
    for e01, e02 in zip(dataset['entity_01'], dataset['entity_02']):
        temp = ''
        temp = e01 + '[SEP]' + e02
        # temp = e01 + '</s></s>' + e02  # roberta
        concat_entity.append(temp)
    tokenized_sentences = tokenizer(
        concat_entity,
        list(dataset['sentence']),
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128,
        add_special_tokens=True,
    )

    return tokenized_sentences

In [5]:
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

In [8]:
tokenized_train = tokenized_dataset(train_dataset, tokenizer)
tokeinzed_val = tokenized_dataset(val_dataset, tokenizer)

In [11]:
tokenized_train_label = train_dataset['label'].values
tokenized_val_label = val_dataset['label'].values

In [9]:
tokenized_train

{'input_ids': tensor([[     0,      6,   7194,  ...,      1,      1,      1],
        [     0,      6,  94320,  ...,      1,      1,      1],
        [     0,  93278, 102384,  ...,      1,      1,      1],
        ...,
        [     0,   5920,  46157,  ...,      1,      1,      1],
        [     0,      6, 128682,  ...,   7063,  56730,      2],
        [     0,  68745,  11758,  ...,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [32]:
train_dataset.iloc[100:110]

,sentence,entity_01,entity_02,label
5766,"강용석 씨는 기자 출신 김세의(43) 씨, 김용호(44) 씨와 유튜브 채널 '가로세...",가세연,기자,0
4245,무엇보다 벤타스 코어는 현대인들이 일상생활 중 가장 많은 피로를 호소하는 허리 부위...,벤타스 코어,허리,0
6092,하지만 1714년 왕위를 계승한 하노버 왕가의 조지 1세가 영어를 잘 구사하지 못했...,조지 1세,하노버 왕가,4
4690,"4월 5일 밤, 김규식, 홍명희, 김붕준 등과 경교장을 찾아 김구 숙소에서 협상 내...",홍명희,경교장,0
8133,"국민의당은 완전 국민 경선 80%, 여론조사 20%가 반영되는 경선을 통해 대한민국...",안철수,국민의당,4
1329,"성주군의 롯데 골프장을 주한미군 기지로 용도변경하여, 사드 1개 포대가 배치되었다.",주한미군 기지,사드,0
8204,"연합뉴스는 ""유서 조작 의혹은 노 전 대통령의 측근이 유서에 담긴 내용이라며 전한 ...",노,대통령,2
1684,의학원은 전문의 및 간호사로 구성된 의료 지원단과 앰뷸런스를 대회 기간 전일 배치해...,LPGA,선수,0
8675,"윌리엄 1세의 지휘하에서, 에옹의 아들 중 3명 (알랑, 에티엔, 브리앙)이 리치먼...",알랑,리치먼드,0
5795,이때 자유당의 실력자 1순위에 올랐던 것은 박마리아의 남편 이기붕과 족청(조선민족청...,조선민족청년단,이범석,10


In [34]:
idx = 105

temp = tokenized_train.input_ids[idx]
print(tokenizer.decode(temp))
print("label: ", tokenized_train_label[idx])

<s> 주한미군 기지[SEP]사드</s></s> 성주군의 롯데 골프장을 주한미군 기지로 용도변경하여, 사드 1개 포대가 배치되었다.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
label:  0


In [ ]:
###### import pickle
import os
import pandas as pd
import torch
from torch.utils.data import Dataset


class TokenDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        tokenized_datasets = self.tokenized_dataset(self.dataset, self.tokenizer)
        item = {key: torch.tensor(val[idx]) for key, val in tokenized_datasets.items()}
        item['labels'] = torch.tensor(list(self.dataset['label'])[idx])
        return item

    def __len__(self):
        return len(self.dataset)
        
    def tokenized_dataset(self, dataset, tokenizer):
        concat_entity = []
        for e01, e02 in zip(dataset['entity_01'], dataset['entity_02']):
            temp = ''
            temp = e01 + '[SEP]' + e02
            # temp = e01 + '</s></s>' + e02  # roberta
            concat_entity.append(taemp)
        
        tokenized_sentences = tokenizer(
            concat_entity,
            list(dataset['sentence']),
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=100,
            add_special_tokens=True,
        )

        return tokenized_sentences

In [106]:
train_dataset

,sentence,entity_01,entity_02,label
6317,무하의 애국심은 히틀러의 독일 정부가 그를의 적으로 간주한 명부에 포함할 만큼 유명했다.,히틀러,독일,7
740,위나라가 퇴각하면서 일대의 주민과 가축을 몽땅 데려가 버렸기 때문에 무도 음평에 주...,촉군,위나라,0
3781,이는 삼성그룹 회장인 이건희가 아들인 이재용에게 경영권을 인계하여 에버랜드-삼성생명...,삼성카드,삼성그룹,3
7850,청원인은 'TJB 대전방송'에서 6년을 근무하며 그중 3년 6개월을 '8뉴스' 앵커...,대전방송,TJB,6
2963,"저명한 교수로는 물리학자 에드워드 빅터 애플턴, 오언 윌런스 리처드슨, 물리학자/수...",찰스 라이엘,지질학,2
...,...,...,...,...
5734,그러나 조지 6세 전 영국 국왕이 초대 수장을 역임한 이래 엘리자베스 2세 현 여왕...,엘리자베스 2세,조지 6세,0
5191,소설은 제2차 세계 대전을 배경으로 가상의 미국 육군 항공대 B-25 폭격기 편대의...,미국 육군 항공대,2차 세계 대전,0
5390,"그는 1975년과 1976년 레즈와 함께 2회의 월드 시리즈를 우승하였고, 각각의 ...",내셔널 리그,1975년,0
860,법무부 장관으로 2014년 신고한 재산내역은 모두 25억8925만원이었으며 법무부 ...,민주노동당,통합진보당,0


In [107]:
train_datasets = TokenDataset(train_dataset, tokenizer)
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
ids = train_datasets[2]['input_ids']
print(train_datasets[2])
tokenizer.decode(ids)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app


{'input_ids': tensor([     0,  93278, 102384,   1065,    294,  21290,    268,  53885,   3276,
        121980,      2,      2,  93172,  93278, 121980,  99862,   1571, 199323,
         23423,    713, 129172,   1571,   1504,   7046,   5358,   9913,  92189,
        119974,   8740,  11931,   8016,  29064,  15164,  65575,      9,  53885,
          3276, 199497,      9,  53885,   3276, 102384,      9,  53885,   3276,
        112242,      9,  29064,  15164,  65575,   1083,  30297,  21252,      6,
        229744,  14842,   2268,    480, 151908,  95026, 216503,    688, 133741,
         92052,  39649,  68682, 126296,      6,  50751,      5,      2,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

'<s> 삼성카드[SEP]삼성그룹</s></s> 이는 삼성그룹 회장인 이건희가 아들인 이재용에게 경영권을 인계하여 에버랜드-삼성생명-삼성카드-삼성전자- 에버랜드로 이어지는 순환출자에 의한 그룹 지배를 확보하는데 있어 중요한 역할을 하였다.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

### K-fold 구현하기
- 가능하면 Stratified

In [108]:
k_folds=5
kfold = KFold(n_splits=k_folds)

In [109]:
kfold

KFold(n_splits=5, random_state=None, shuffle=False)

In [110]:
for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset)):
        print("train: ", train_ids)
        print("val: ", val_ids)
        
        temp = dataset.iloc[train_ids]
#         print("temp: ", temp)

train:  [1800 1801 1802 ... 8997 8998 8999]
val:  [   0    1    2 ... 1797 1798 1799]
train:  [   0    1    2 ... 8997 8998 8999]
val:  [1800 1801 1802 ... 3597 3598 3599]
train:  [   0    1    2 ... 8997 8998 8999]
val:  [3600 3601 3602 ... 5397 5398 5399]
train:  [   0    1    2 ... 8997 8998 8999]
val:  [5400 5401 5402 ... 7197 7198 7199]
train:  [   0    1    2 ... 7197 7198 7199]
val:  [7200 7201 7202 ... 8997 8998 8999]


### Trainer 안쓰고 trainingmax_len=128

In [125]:
max_len=128
batch_size=32
warmup_ratio=0.01
num_epochs=10
max_grad_norm=1
log_interval=50
learning_rate=5e-5

In [127]:
import torch.nn as nn

class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes=42, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [129]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_config = AutoConfig.from_pretrained('xlm-roberta-large')
model_config.num_labels = 42
model = AutoModelForSequenceClassification.from_pretrained('xlm-roberta-large', config=model_config)

model.parameters
model.to(device)

train_loader = DataLoader(train_datasets, batch_size=32, shuffle=True)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.den

In [130]:
from transformers.optimization import get_cosine_schedule_with_warmup

optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = LabelSmoothingLoss()

t_total = len(train_loader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [132]:
next(iter(train_loader))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app


{'input_ids': tensor([[     0, 203559,   7849,  ...,      1,      1,      1],
         [     0,  21486,    480,  ...,      1,      1,      1],
         [     0,   8237,  23224,  ...,      1,      1,      1],
         ...,
         [     0,   7463,   3013,  ...,      1,      1,      1],
         [     0,     15,   2688,  ...,      1,      1,      1],
         [     0,   1504,  20126,  ...,      1,      1,      1]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([ 0,  0,  1,  2,  0, 20,  0, 15,  0,  2,  0,  4,  0, 10,  0, 15, 15, 15,
          0,  0,  0,  0,  0,  4,  0,  9,  0,  0,  2,  0, 10,  0])}

In [124]:
nlp.data.BERTSentenceTransform(tokenizer, max_seq_length=128, pad=True, pair=True)

AttributeError: 'dict' object has no attribute 'bos_token'

In [134]:


for epoch in range(3):
    for batch in tqdm(train_loader):
        print(batch)
        print("epoch!")
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

  0%|          | 0/225 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app


{'input_ids': tensor([[     0,   8511,  12431,  ...,      1,      1,      1],
        [     0,      6, 231016,  ...,      1,      1,      1],
        [     0,      6, 132467,  ...,      1,      1,      1],
        ...,
        [     0,   3626,  13902,  ...,      1,      1,      1],
        [     0,   8237,  26870,  ...,      1,      1,      1],
        [     0,  14592,  27082,  ...,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([ 4, 10, 15, 10,  0,  6,  8,  0,  0,  0, 25,  0,  0,  0,  2, 10, 17, 10,
         0,  0,  0,  0,  2, 10,  0,  0,  0,  0, 35,  0, 27,  4])}
epoch!


  0%|          | 1/225 [00:19<1:11:30, 19.16s/it]

{'input_ids': tensor([[     0,   7592,  12412,  ...,      1,      1,      1],
        [     0,      6, 217783,  ...,      1,      1,      1],
        [     0,  20294,   5821,  ...,   5476,      5,      2],
        ...,
        [     0, 128300,   1065,  ...,      1,      1,      1],
        [     0,   5241,   8443,  ...,      1,      1,      1],
        [     0,  41716,   1083,  ...,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([ 0, 10,  0, 10, 13,  0,  2,  0,  4,  0,  0,  0,  4,  2,  0,  4,  0,  0,
         0,  0,  0, 24, 29,  0,  2,  0,  0,  0, 22,  0, 36, 26])}
epoch!


  1%|          | 2/225 [00:47<1:28:51, 23.91s/it]


KeyboardInterrupt: 